# Make Recall Vectors
## 12/3/20
## Make a dict, where for each story there is another dict with keys: 0, 1, 2, containing wvs for each participant
## 1/21/21 - adding more information about each recall

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random

from sklearn.decomposition import PCA #for cluster analysis
from gensim.models import KeyedVectors #for word embeddings
import matplotlib.pyplot as plt #for plotting
import os #for importing
import pickle #for loading transcripts
from scipy.stats import pearsonr 

# from _DRAFT_20200604_functions import * #includes constants and score function
from tqdm import tqdm_notebook #for progress bar

%autosave 5

Autosaving every 5 seconds


# I. Imports
## A. Participant Data and Summary of Data

In [2]:
filename = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/analysis/segment/usable_data_recall2'
filename_sums = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/analysis/segment/sums_usable_data_recall2'
data_dict = pickle.load( open( filename, "rb" ) )
sums = pickle.load( open( filename_sums, "rb" ) )
len(data_dict)

379

## B. Importing wiki vectors

In [3]:
wikipath = 'rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

# II. Functions

In [4]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return wv

## III. Making the story vectors
### A. Organize free recalls in dict
#### dict[story]{0: [np recalls]; 1: [l recalls]; 2: [s recalls]}

In [5]:
recalls = {}

#get the key and prolific id 

for key in data_dict:
    # Making a column with prolific id in first row and recall in second
    this_partc = np.zeros((1,3), dtype=object)
    this_partc[0] = data_dict[key]['recall'][0]
    this_partc[0,1] = key
    this_partc[0,2] = data_dict[key]['prolific_id'][0]
    # Naming this story
    this_story = data_dict[key]['story'][0]
    # Naming this perspective
    this_persp = data_dict[key]['p_type'][0]
    if isinstance(this_partc[0][0], float):
        print(key, '\nthis story:', this_story, "this_persp: ", this_persp)
        pass
    else:
        if this_story in recalls:
            if this_persp in recalls[this_story]:
                recalls[this_story][this_persp] = np.vstack((recalls[this_story][this_persp],this_partc))
            else:
                recalls[this_story][this_persp] = this_partc
        else:
            recalls[this_story] = {this_persp: this_partc}

In [6]:
n = 0
for key in recalls:
    for i in range(0,3):
        n += recalls[key][i].shape[0]
        
print(n)

379


In [7]:
recalls

{44: {0: array([["The guy walks into the lecture hall and sees Ariana, and immediately feels like she's very familiar. They all sit down, the professor starts talking about mummification, meanwhile the two start exchanging notes. He asks her if she's from LA, because she looks very familiar. I do not remember how she replies, but this goes on until he asks her to meet the evening for a walk, which she agrees by looking at him and nodding. Meanwhile the professor still teaches his lesson. The period ends, the professor speaks of the assignment about mummification and everyone leaves except the two. I do not remember what they say, except that the guy's conviction is even stronger after he hears her voice.",
          '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h54.27.962',
          '5cdeaeb3557fbd00018a2c59'],
         ['Leon found a girl he tought he already met before. In 

## C. Export the free recalls and sums

In [8]:
pickle.dump( recalls, open( 'fr_recalls2', "wb" ) )
pickle.dump( sums, open( 'fr_sums2', "wb" ) )

### B. Getting Word Vectors for each free recall 
#### Output: dict w same structure as recalls

In [9]:
keys = [11, 12, 13, 14, 21, 22, 23, 24, 31, 32, 33, 34, 41, 42, 43, 44]

wv_matrices = {}
keys_ids = np.zeros((0,2))
ids = {}

# Iterating through each key
for key in keys:
    print(key)
    # Iterating through each priming type
    for i in range(0,3):
        # Get the number of participants in that priming group for that story
        partic_n = len(recalls[key][i])
        # Make the matrix containing the word vectors for each participants' recall
        wv_matrix = np.zeros((partic_n, wv_dim))
        # Make the matrix with the ids
        id_matrix = np.zeros((0,2))
        for j in range(0, partic_n):
            # Get the recall for that participant
            this_recall = recalls[key][i][j][0]
            # Get the key
            this_key = recalls[key][i][j][1]
            # Get the prolific id
            this_id = recalls[key][i][j][2]
            # Get the wv for that recall
            this_vector = np.mean(word2vecSent(this_recall), axis = 0)
            # Add wv to matrix
            wv_matrix[j, :] = this_vector
            # Create matrix with this_key and this_id
            this_matrix = np.array([this_key, this_id])
            # Add key and prolific id to keys_ids matrix
            keys_ids = np.vstack((keys_ids, this_matrix))
            id_matrix = np.vstack((id_matrix, this_matrix))
        if key in wv_matrices:
            wv_matrices[key][i] = wv_matrix
            ids[key][i] = id_matrix
        else:
            wv_matrices[key] = {i: wv_matrix}
            ids[key] = {i: id_matrix}

11
12
13
14
21
22
23
24
31
32
33
34
41
42
43
44


### C. Export Word Vector Matrices and sums

In [10]:
pickle.dump( wv_matrices, open( 'recall_vectors', "wb" ) )
pickle.dump( sums, open( 'new_sums', "wb" ) )
pickle.dump( keys_ids, open( 'ids_recall', "wb" ) )
# ids organized by story and priming
pickle.dump( ids, open( 'ids_dict', "wb" ) )
# data_dict to access recalls in other notebooks
pickle.dump( data_dict, open( 'data_dict', "wb" ) )